# Web Scraping solution


## Step 1: Install dependencies

In [5]:
pip install pandas requests lxml

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Step 2: Download HTML

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import io

url = "https://en.wikipedia.org/wiki/List_of_Spotify_streaming_records"

headers = {
    "User-Agent": "Mozilla/5.0 (compatible; DataScienceProject/1.0; +https://example.com)"
}

response = requests.get(url, headers=headers)

response.raise_for_status()

print("Status:", response.status_code)

Status: 200


## Step 3: Transform the HTML

In [8]:
# Extract tables with pandas
html = io.StringIO(response.text)  # Convert the HTML to a text file

# read_html() returns a list of DataFrames
tables = pd.read_html(html)
print(f"{len(tables)} tables were found.")

26 tables were found.


In [9]:
# Inspect the first rows of the correct table
df = tables[0]  # Extract the first table from the 27 found
df.head()  # Display the first 5 rows

,Rank,Song,Artist(s),Streams (billions),Release date,Ref.
0,1,"""Blinding Lights""",The Weeknd,5.279,29 November 2019,[1]
1,2,"""Shape of You""",Ed Sheeran,4.771,6 January 2017,[2]
2,3,"""Sweater Weather""",The Neighbourhood,4.384,3 December 2012,[3]
3,4,"""Starboy""",The Weeknd and Daft Punk,4.367,21 September 2016,[4]
4,5,"""As It Was""",Harry Styles,4.250,1 April 2022,[5]


## Step 4: Process the DataFrame

In [10]:
# Data cleaning

# Rename columns if necessary
df.columns = ["Rank", "Song", "Artist", "Streams (billions)", "Date released", "Reference"]

df = df.copy()  # Create a copy of the DataFrame

# Remove notes in square brackets
df["Song"] = df["Song"].str.replace(r"\[.*?\]", "", regex=True)
df["Artist"] = df["Artist"].str.replace(r"\[.*?\]", "", regex=True)

# Filter rows containing a valid number in the "Streams (billions)" column
df = df[df["Streams (billions)"].astype(str).str.contains(r"^\d+(?:\.\d+)?$", na=False)].copy()

# Convert Streams to float numbers
df["Streams (billions)"] = df["Streams (billions)"].astype(float)

# Convert dates to datetime
df["Date released"] = pd.to_datetime(df["Date released"], errors="coerce")

df

,Rank,Song,Artist,Streams (billions),Date released,Reference
0,1,"""Blinding Lights""",The Weeknd,5.279,2019-11-29,[1]
1,2,"""Shape of You""",Ed Sheeran,4.771,2017-01-06,[2]
2,3,"""Sweater Weather""",The Neighbourhood,4.384,2012-12-03,[3]
3,4,"""Starboy""",The Weeknd and Daft Punk,4.367,2016-09-21,[4]
4,5,"""As It Was""",Harry Styles,4.250,2022-04-01,[5]
...,...,...,...,...,...,...
95,96,"""Payphone""",Maroon 5 and Wiz Khalifa,2.553,2012-04-16,[96]
96,97,"""Dreams""",Fleetwood Mac,2.547,1977-02-04,[97]
97,98,"""Sicko Mode""",Travis Scott and Drake,2.533,2018-08-21,[98]
98,99,"""Billie Jean""",Michael Jackson,2.525,1982-11-29,NaN


## Step 5: Store the data in sqlite

In [ ]:
# Create the database
conn = sqlite3.connect("spotify_top_songs.db")

In [ ]:
# Create table in SQLite
df.to_sql("most_streamed", conn, if_exists="replace", index=False)
cursor = conn.cursor()

In [ ]:
# Insert data into the database
cursor.execute("SELECT COUNT(*) FROM most_streamed")
print("Rows inserted:", cursor.fetchone()[0])

conn.commit()
conn.close()

## Step 6: Visualize the data

### Visual Analysis of Popularity on Spotify

In [ ]:
# Plot the data

# Chart 1: Top 10 most-streamed songs
top10 = df.nlargest(10, "Streams (billions)")
plt.figure(figsize=(12, 6))
sns.barplot(data=top10, x="Streams (billions)", y="Song", hue="Song", palette="viridis", legend=False)
plt.title("Top 10 Most-Streamed Songs on Spotify")
plt.xlabel("Streams (in billions)")
plt.ylabel("Song")
plt.tight_layout()
plt.show()

In [ ]:
# Chart 2: Number of songs per year
df["Year"] = df["Date released"].dt.year
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x="Year", order=sorted(df["Year"].dropna().unique()))
plt.title("Number of Songs in the Ranking by Release Year")
plt.xlabel("Year")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Chart 3: Artists with the most songs in the ranking

# Note: Some artists have multiple songs
artists = df["Artist"].value_counts().nlargest(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=artists.values, y=artists.index, hue=artists.index, palette="coolwarm", legend=False)
plt.title("Artists with the Most Songs in the Ranking")
plt.xlabel("Number of Songs")
plt.ylabel("Artist")
plt.tight_layout()
plt.show()